## Decompose the Ask

Have customers narrow their travel searches based on temperature and precipitation

## Identify the Data Sources

Use WeatherPy_vacation.csv file from Part 1 of the Challenge

## Data Retrieval

In [1]:
# Import the dependencies (packages required to run this notebook).
# Import pandas to read the CSV file and create locations and measurements from the DataFrame
import pandas as pd

# Import gmaps and the API key to create heatmaps and the locations map
import gmaps

from config import g_key

# Import requests dependency to make a request to the Google Places JSON file
import requests

In [2]:
# Import the CSV you created in part one of the Challenge into a DataFrame.
part_one_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
part_one_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Chuy,UY,2020-02-23 18:32:50,-33.70,-53.46,71.78,59,66,4.61,broken clouds,0.0,0.0
1,1,Vao,NC,2020-02-23 18:28:52,-22.67,167.48,77.65,76,83,14.32,broken clouds,0.0,0.0
2,2,Bredasdorp,ZA,2020-02-23 18:32:51,-34.53,20.04,66.20,59,0,12.75,clear sky,0.0,0.0
3,3,Itarema,BR,2020-02-23 18:32:51,-2.92,-39.92,85.75,69,85,4.65,light rain,0.0,0.0
4,4,Lebu,CL,2020-02-23 18:32:02,-37.62,-73.65,69.19,57,2,9.35,clear sky,0.0,0.0


In [4]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_pref = str(input("Do you want it to be raining? (yes/no) "))
snow_pref = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [7]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = part_one_df.loc[(part_one_df["Max Temp"] <= max_temp) & \
                                       (part_one_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
1,1,Vao,NC,2020-02-23 18:28:52,-22.67,167.48,77.65,76,83,14.32,broken clouds,0.0,0.0
3,3,Itarema,BR,2020-02-23 18:32:51,-2.92,-39.92,85.75,69,85,4.65,light rain,0.0,0.0
7,7,Puerto Ayora,EC,2020-02-23 18:33:35,-0.74,-90.35,84.20,74,20,12.75,few clouds,0.0,0.0
8,8,Rikitea,PF,2020-02-23 18:33:35,-23.12,-134.97,78.37,74,2,8.25,light rain,0.0,0.0
10,10,Biltine,TD,2020-02-23 18:32:53,14.53,20.92,77.99,11,0,11.72,clear sky,0.0,0.0
11,11,Cayenne,GF,2020-02-23 18:33:36,4.93,-52.33,84.20,70,75,14.99,broken clouds,0.0,0.0
19,19,San Andres,CO,2020-02-23 18:29:23,12.58,-81.70,84.20,66,40,6.93,scattered clouds,0.0,0.0
21,21,Foso,GH,2020-02-23 18:33:40,5.70,-1.29,85.39,71,100,7.00,overcast clouds,0.0,0.0
22,22,Marawi,PH,2020-02-23 18:33:06,8.00,124.29,75.00,85,38,5.01,scattered clouds,0.0,0.0
24,24,Poum,NC,2020-02-23 18:33:43,-20.23,164.02,80.62,76,37,22.37,scattered clouds,0.0,0.0


In [12]:
# Iterate through the DataFrame.
for index, row in preferred_cities_df.iterrows():
    info_box_template = """
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
    </dl>
    """
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in preferred_cities_df.iterrows()]

KeyError: 'Hotel Name'

In [13]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = preferred_cities_df[["Lat", "Lng"]]
max_temp = preferred_cities_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
 #            max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

NameError: name 'hotel_info' is not defined